# Text Generation Using RNNs for D&D Applications

In this notebook, we use the Keras_Text_Generator from the textGen module to train a GRU on text data generated from Dungeons and Dragons monster statblocks. Our goal is to create new statblocks for plausible/usable monsters using a recurrent neural network.

First thing to do is import necessary modules:

In [0]:
from textGen import Keras_Text_Generator
from tensorflow.keras.layers import Embedding, GRU, Dropout, Dense
import tensorflow as tf
#!pip install tensorflow-gpu==2.0.0-beta1

Next we instantiate a text generator object that will hold our data and model and then finally output our generated text.

In [0]:
gen = Keras_Text_Generator()

We are working with a large string that represents all of the monsters currently available in the latest edition of Dungeons and Dragons simply strung together in one continuous piece of text. Let's take a look at what we're working with.

In [3]:
with open('adjusted_full_abilities.txt', 'rb') as f:
  print(f.read()[:100])

b'<<start>>[[challenge]]1/4 {50 xp}[[strength]]10 |0|[[dexterity]]14 |+2|[[constitution]]10 |0|[[wisdo'


We now call the load_and_create_dataset method which will load from the designated file and vectorize our data, as well as store it into a Tensorflow Dataset object that we will use to iterate over. We'll choose a sequence length of 500, which will create training instances that are 500 characters in length. 


I'll set the rolling sequences flag to True as we'll want to create as much data as we can. Setting this flag instructs the method to create data essentially by using a rolling window with the initial training instance consisting of the first seq_length number of characters and the second training instance starting on the second character and consisting of seq_length characters and so on. If we set this flag to false, we'd simply chop the data into seq_length size chunks.


We'll go with a sequence length of 500 because there is a balance to be struck between creating training instances long enough for the GRU units to recognize dependencies in statblocks that can be somewhat lengthy, however we also need to think about memory cost. There are ~840,000 characters in our text string, so this will create a dataset that is a 500 x 840,000 matrix which can weigh down RAM resources. However, testing shows that a larger seq_length can be beneficial for results.

In [4]:
gen.load_and_create_dataset('adjusted_full_abilities.txt', seq_length=500, rolling_sequences=True, 
                            rolling_sequences_step=3)

Length of text: 841581 characters
Unique characters: 67
Dataset successfully created.


Next, to construct the model, we'll add each layer one at a time using the add_layer_to_model method, and by specifying the layer along with its keyword arguments.

In [0]:
vocab_size = gen.vocab_size
embedding_dim = 200
gen.add_layer_to_model(Embedding, 
                       input_dim=vocab_size, 
                       output_dim=embedding_dim)
gen.add_layer_to_model(GRU,
                       units=300,
                       return_sequences=True,
                       stateful=True,
                       recurrent_initializer='glorot_uniform')
gen.add_layer_to_model(Dropout,
                      rate=0.1)
gen.add_layer_to_model(GRU,
                       units=300,
                       return_sequences=True,
                       stateful=True,
                       recurrent_initializer='glorot_uniform')
gen.add_layer_to_model(Dropout,
                      rate=0.1)
gen.add_layer_to_model(Dense,
                      units=vocab_size)

As you can see below, once we compile the model, there are quite a few trainable parameters. Memory units can be very compute intensive.

In [6]:
gen.compile_model()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 200)           13400     
_________________________________________________________________
gru (GRU)                    (64, None, 300)           451800    
_________________________________________________________________
dropout (Dropout)            (64, None, 300)           0         
_________________________________________________________________
gru_1 (GRU)                  (64, None, 300)           541800    
_________________________________________________________________
dropout_1 (Dropout)          (64, None, 300)           0         
_________________________________________________________________
dense (Dense)                (64, None, 67)            20167     
Total params: 1,027,167
Trainable params: 1,027,167
Non-trainable params: 0
______________________________________________

Now we're ready to train the model. Let's call the fit_model method and see what kind of improvement in loss we can get. From experience, categorical cross entropy loss usually starts at around 4.0 before training.

In [7]:
gen.fit_model(2)

Epoch 1/2
4380/4380 [==============================] - 1596s 364ms/step - loss: 0.3503
Epoch 2/2
4380/4380 [==============================] - 1588s 363ms/step - loss: 0.1671


Next, we load the model, which allows us to reset the input dimensions that the model should expect. This is important because before we were using batches and the model was looking for batch_size inputs. Now however, we want to simply feed in a single text string to represent the starting sequence for our text generator. Let's take a look - the model should load with the same architecture it was trained on:

In [8]:
gen.load_model_from_checkpoint()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 200)            13400     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 300)            451800    
_________________________________________________________________
dropout_2 (Dropout)          (1, None, 300)            0         
_________________________________________________________________
gru_3 (GRU)                  (1, None, 300)            541800    
_________________________________________________________________
dropout_3 (Dropout)          (1, None, 300)            0         
_________________________________________________________________
dense_1 (Dense)              (1, None, 67)             20167     
Total params: 1,027,167
Trainable params: 1,027,167
Non-trainable params: 0
____________________________________________

Looks like everything is in order. Now to get down to business. Let's generate some text using the generate_text method. We have a temperature keyword argument that we can use to control the probability distribution that we are sampling from to generate the output ASCII characters. Anything above 1.0 should generate more surprising characters, and anything less would generate less surprising characters. Let's see what we get!

In [10]:
gen.generate_text(temperature=0.8)

"<<start>>[[challenge]]3 {700 xp}[[strength]]11 |0|[[dexterity]]18 |+4|[[constitution]]16 |+3|[[wisdom]]13 |+1|[[intelligence]]11 |0|[[charisma]]10 |0|[[armor_class]]16 (studded leather armor)[[hit_points]]75 (10d8+30)[[speed]]30 ft.[[damage_immunities]]none[[damage_resistances]]none[[condition_immunities]]none[[saving_throws]]none[[short_desc]]medium humanoid (any race), any alignment[[skills]]acrobatics +6, perception +5[[languages]]any one language (usually common)[[full_ability]]spellcasting;; the acolyte is a 1st-level spellcaster. its spellcasting ability is wisdom (spell save dc 12, +4 to hit with spell attacks). the acolyte has following cleric spells prepared:cantrips (at will): light, sacred flame, thaumaturgy+7 to hit, reach 5 ft., one target. hit: 1 (1d4 - 1) slashing damage.<<end>><<start>>[[challenge]]00[[strength]]16 |+3|[[dexterity]]16 |+3|[[constitution]]16 |+3|[[wisdom]]17 |+3|[[intelligence]]14 |+2|[[charisma]]18 |+4|[[armor_class]]29 (natural armor)[[hit_points]]333

Success! We have complete monsters generated by the RNN! The sequence that it generated is remarkably similar to a normal D&D monster statblock. It was able to pick up the structure of ability_name: ability_score pairs for strength, dexterity, constitution, wisdom, intelligence, charisma, etc. 

The curious reader will note that I set the temperature to 0.8, which means that these results should be less surprising. Without double-checking, I would be willing to guess that the monsters generated above are actually close copies of existing monsters which is less that desirable. Let's try with an increased temperature.

In [11]:
gen.generate_text(temperature=1.2)

'<<start>>[[challenge]]2 {450 xp}[[strength]]16 |+4|[[dexterity]]13 |+1|[[constitution]]14 |+2|[[wisdom]]11 |0|[[intelligence]]14 |+2|[[charisma]]11 |0|[[armor_class]]13[[hit_points]]20 (4d10+11)[[speed]]40 ft., burrow 5 ft.[[damage_immunities]]none[[damage_resistances]]none[[condition_immunities]]none[[saving_throws]]none[[short_desc]]large beast, unaligned[[skills]]none[[languages]]none[[full_ability]]tail;; melee weapon attack: +7 to hit, reach 10 ft., one target. hit: 18 (4d6 + 4) bludgeoning damage. if the target is a creature, it must succeed on a dc 15 strength saving throw or be knocked prone.<<end>><<start>>[[challenge]]2 {450 xp}[[strength]]17 |+3|[[dexterity]]11 |0|[[constitution]]13 |+1|[[wisdom]]13 |+1|[[intelligence]]1 |-5|[[charisma]]6 |-2|[[armor_class]]14 (natural armor, 11 while prone)[[hit_points]]39 (6d10+6)[[speed]]30 ft., burrow 15 ft., swim 40 ft.[[damage_immunities]]none[[damage_resistances]]none[[condition_immunities]]none[[saving_throws]]none[[short_desc]]medi

The higher temperature is still surprisingly true to the structure of a statblock. However, upon further examination, the string has some skips where the generator samples a "more surprising" character which feel more "out of sequence" than surprising and the GRU snaps into a new sequence seemingly all of a sudden. I would describe this as a mistake. When this happens, the statblock is not more surprising, but rather unusable. This marks an area for improvement in the future.

## Final Thoughts

We got some truly fantastic results. The RNNs were able to very closely mimic the strange syntax of a D&D statblock. It was even able to accomplish dice math with a relatively high degree of reliability when determining statistics like hit points. And did so without any mathematical formulas!

We did discover that we have a little bit of a catch-22 when generating text: when we set the temperature lower, we find near copies of existing statblocks, but when we set it higher, we risk the generator breaking syntax. We could continue to try and solve this problem with hyperparameter tuning as well as different variations on the neural network's architecture. Another solution might be to try using word level generators. This would allow for transfer learning, but would potentially limit the vocabulary of the generator. Regardless, with recent advances in transfer learning in NLP, it seems to demand a test. 

More to come! Stay tuned!